<a href="https://colab.research.google.com/github/Tharungowdapr/computer-vision-/blob/main/Plant%20village%20vit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SETUP PYTORCH + LIBRARIES IN COLAB**

In [2]:
!pip install kagglehub -q


In [3]:
import kagglehub

# Download latest version of the PlantVillage dataset
path = kagglehub.dataset_download("emmarex/plantdisease")

print("Dataset downloaded to:", path)


Using Colab cache for faster access to the 'plantdisease' dataset.
Dataset downloaded to: /kaggle/input/plantdisease


In [4]:
import os

print(os.listdir(path))


['PlantVillage', 'plantvillage']


In [5]:
print(os.listdir(f"{path}/PlantVillage"))


['Pepper__bell___Bacterial_spot', 'Potato___healthy', 'Tomato_Leaf_Mold', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Bacterial_spot', 'Tomato_Septoria_leaf_spot', 'Tomato_healthy', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_Early_blight', 'Tomato__Target_Spot', 'Pepper__bell___healthy', 'Potato___Late_blight', 'Tomato_Late_blight', 'Potato___Early_blight', 'Tomato__Tomato_mosaic_virus']


In [6]:
import os, shutil, random
from glob import glob

source_dir = f"{path}/PlantVillage"
target_dir = "/content/plantvillage_split"

def create_split(src, dst, split=0.2):
    classes = os.listdir(src)

    for cls in classes:
        os.makedirs(f"{dst}/train/{cls}", exist_ok=True)
        os.makedirs(f"{dst}/val/{cls}", exist_ok=True)

        imgs = glob(f"{src}/{cls}/*")
        random.shuffle(imgs)

        val_count = int(len(imgs) * split)

        for img in imgs[val_count:]:
            shutil.copy(img, f"{dst}/train/{cls}/")

        for img in imgs[:val_count]:
            shutil.copy(img, f"{dst}/val/{cls}/")

create_split(source_dir, target_dir)

print("Split created at:", target_dir)


Split created at: /content/plantvillage_split


In [7]:
!pip install timm -q


In [8]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

def make_loaders(path, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,)*3, (0.5,)*3)
    ])

    train_data = datasets.ImageFolder(f"{path}/train", transform)
    val_data   = datasets.ImageFolder(f"{path}/val", transform)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_data, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, len(train_data.classes)


In [9]:
import timm
import torch.nn as nn
from torch.optim import Adam

train_loader, val_loader, classes = make_loaders(target_dir)

model = timm.create_model(
    "vit_base_patch16_224",
    pretrained=True,
    num_classes=classes
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=3e-5)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [10]:
from tqdm import tqdm

def train_epoch():
    model.train()
    total, correct, loss_sum = 0, 0, 0

    for x, y in tqdm(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        _, pred = out.max(1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    return loss_sum / len(train_loader), 100 * correct / total


def val_epoch():
    model.eval()
    total, correct, loss_sum = 0, 0, 0

    with torch.no_grad():
        for x, y in tqdm(val_loader):
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)

            loss_sum += loss.item()
            _, pred = out.max(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    return loss_sum / len(val_loader), 100 * correct / total


In [11]:
epochs = 5

for epoch in range(epochs):
    t_loss, t_acc = train_epoch()
    v_loss, v_acc = val_epoch()

    print(f"\nEpoch {epoch+1}/{epochs}")
    print(f"Train Loss: {t_loss:.4f} | Train Acc: {t_acc:.2f}%")
    print(f"Val   Loss: {v_loss:.4f} | Val   Acc: {v_acc:.2f}%")


100%|██████████| 129/129 [00:55<00:00,  2.34it/s]



Epoch 1/5
Train Loss: 0.1655 | Train Acc: 94.88%
Val   Loss: 0.0393 | Val   Acc: 98.88%


100%|██████████| 129/129 [00:55<00:00,  2.34it/s]



Epoch 2/5
Train Loss: 0.0230 | Train Acc: 99.30%
Val   Loss: 0.0332 | Val   Acc: 98.98%


100%|██████████| 129/129 [00:55<00:00,  2.34it/s]



Epoch 3/5
Train Loss: 0.0241 | Train Acc: 99.20%
Val   Loss: 0.0209 | Val   Acc: 99.39%


100%|██████████| 129/129 [00:55<00:00,  2.33it/s]



Epoch 4/5
Train Loss: 0.0121 | Train Acc: 99.58%
Val   Loss: 0.0633 | Val   Acc: 98.11%


100%|██████████| 129/129 [00:55<00:00,  2.32it/s]


Epoch 5/5
Train Loss: 0.0158 | Train Acc: 99.50%
Val   Loss: 0.0387 | Val   Acc: 98.81%


In [12]:
torch.save(model.state_dict(), "vit_plantvillage.pth")
print("Model saved!")


Model saved!
